<a href="https://colab.research.google.com/github/Thabet-Chaaouri/Transformers/blob/main/Fine_Tune_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install sacrebleu
!pip install evaluate
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 64.7 MB/s 
     |████████████████████████████████| 182 kB 63.4 MB/s 
     |████████████████████████████████| 1.3 MB 66.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 61.2 MB/s 
     |████████████████████████████████| 132 kB 69.3 MB/s 
     |████████████████████████████████| 127 kB 72.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 5.1 MB/s 
Looking in indexes: https

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("kde4", lang1="en", lang2="fr")

Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

Dataset kde4 downloaded and prepared to /root/.cache/huggingface/datasets/kde4/en-fr-lang1=en,lang2=fr/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(f"nb of examples {raw_dataset.num_rows}")

nb of examples {'train': 210173}


In [ ]:
split_datasets=raw_dataset["train"].train_test_split(test_size=0.1, seed=20)
split_datasets["validation"]=split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [ ]:
from transformers import AutoTokenizer

checkpoint ="Helsinki-NLP/opus-mt-fr-en"
tokenizer = AutoTokenizer.from_pretrained(checkpoint,)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/802k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
example=tokenizer(split_datasets["train"][1]["translation"]["en"],text_target=split_datasets["train"][1]["translation"]["fr"], )
example

{'input_ids': [333, 483, 8153, 12, 17715, 18074, 111, 7532, 9832, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [577, 1205, 483, 13077, 2, 1205, 517, 13926, 1588, 16, 3842, 9, 5, 1710, 0]}

In [ ]:
max_length=128

def preprocess_data(examples):
  inputs=[example["en"] for example in examples["translation"]]
  targets=[example["fr"] for example in examples["translation"]]
  return tokenizer(inputs, text_target=targets, truncation=True, max_length=max_length)

In [ ]:
tokenized_datasets = split_datasets.map(preprocess_data, batched=True, remove_columns=split_datasets["train"].column_names,)
tokenized_datasets

  0%|          | 0/190 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/301M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-fr-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [ ]:
import numpy as np
import tensorflow as tf
import tqdm as tqdm
from transformers import DataCollatorForSeq2Seq

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

sampled_dataset = tokenized_datasets["validation"].shuffle().select(range(200))
tf_generate_dataset = model.prepare_tf_dataset(sampled_dataset, collate_fn=generation_data_collator, shuffle=False, batch_size=8)

In [ ]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
  return model.generate(input_ids=batch["input_ids"], attention_mask = batch["attention_mask"], max_new_tokens=128)

def compute_metrics():
    all_preds = []
    all_labels = []

    for batch, labels in tqdm.tqdm(tf_generate_dataset):
        predictions = generate_with_xla(batch)
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels.numpy()
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)

    result = metric.compute(predictions=all_preds, references=all_labels)
    return {"bleu": result["score"]}

In [ ]:
compute_metrics()

100%|██████████| 25/25 [01:22<00:00,  3.28s/it]


{'bleu': 8.539345350158822}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

tf_train_dataset = model.prepare_tf_dataset(tokenized_datasets["train"], collate_fn=data_collator, shuffle=True, batch_size=32)
tf_val_dataset = model.prepare_tf_dataset(tokenized_datasets["validation"], collate_fn=data_collator, shuffle=False, batch_size=32)

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback

num_epochs = 3
num_train_steps = len(tf_train_dataset)*num_epochs

optimizer, schedule = create_optimizer(init_lr = 1e-5, num_warmup_steps = 10, num_train_steps = num_train_steps, weight_decay_rate=0.01)

model.compile(optimizer=optimizer)

tf.keras.mixed_precision.set_global_policy("mixed_float16")

callback = PushToHubCallback(output_dir="marian_finetuned_kde4_en_fr", tokenizer=tokenizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
Cloning https://huggingface.co/Thabet/marian_finetuned_kde4_en_fr into local empty directory.


In [ ]:
model.fit(tf_train_dataset, validation_data = tf_val_dataset, callbacks=[callback], epochs=num_epochs)

Epoch 1/3
5911/5911 [==============================] - ETA: 0s - loss: 1.8240

Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


5911/5911 [==============================] - 4487s 755ms/step - loss: 1.8240 - val_loss: 1.2840
Epoch 2/3
5911/5911 [==============================] - 4431s 750ms/step - loss: 1.3011 - val_loss: 1.1332
Epoch 3/3
5911/5911 [==============================] - 4458s 754ms/step - loss: 1.1889 - val_loss: 1.0969


In [ ]:
compute_metrics()